# Load packages

In [1]:
import numpy as np
import pandas as pd
from functions.standardised_difference import standardised_difference_function
np.random.seed(42)

# Load Data

In [2]:
df = pd.read_csv("./data/iris.csv")
df['group_var'] = np.random.choice(a=[0, 1],  size=len(df),  p=[0.725,0.275])
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,group_var
0,5.1,3.5,1.4,0.2,Iris-setosa,0
1,4.9,3.0,1.4,0.2,Iris-setosa,1
2,4.7,3.2,1.3,0.2,Iris-setosa,1
3,4.6,3.1,1.5,0.2,Iris-setosa,0
4,5.0,3.6,1.4,0.2,Iris-setosa,0


# Define global variables

In [3]:
treatment_control_flag = 'group_var'
var_name = 'Species'
control_label = 0
treatment_label = 1

# Calculate ColPercent (as in SAS macro)

In [5]:
# For the favoured class
control = df.loc[df[treatment_control_flag] == control_label, [treatment_control_flag, var_name]].rename(columns = {treatment_control_flag: 'control'})
n_control = len(control)
control = control.groupby(control[var_name]).count()/len(control)
# For the deprived class
treatment = df.loc[df[treatment_control_flag] == treatment_label, [treatment_control_flag, var_name]].rename(columns = {treatment_control_flag: 'treatment'})
n_treatment = len(treatment)
treatment = treatment.groupby(treatment[var_name]).count()/len(treatment)
# Merge both results in a single table as suggested by  Dongsheng, Y. and Dalton, E (2012)
ColPercent = pd.merge(control, treatment, on = var_name)
ColPercent 

,control,treatment
Species,,
Iris-setosa,0.366972,0.243902
Iris-versicolor,0.302752,0.414634
Iris-virginica,0.330275,0.341463


# Calculate COV matrix S

In [7]:
# Get the number of K for the construction of the COV matrix. K = num of classes
k = len(np.unique(df[var_name]))
m = k - 1
ColPercent = ColPercent.iloc[:m]
# Clear indexes 
temp = ColPercent.reset_index(drop=True)

# Calculate a (m − 1) × (m − 1) covariance matrix S
s = []
for i in range(m):
    a = []
    for j in range(m):
        if i == j:
            val =  0.5 * (temp['treatment'][i] * (1 - temp['treatment'][i]) + temp['control'][i] * (1 - temp['control'][i]))                  
        else:
            val = -0.5 * (temp['treatment'][i] * temp['treatment'][j] + temp['control'][i] * temp['control'][j])   
        a.append(val)
    s.append(a)

S = np.linalg.inv(np.array(s))
S = np.matrix(S)
# Inverted matrix
S

matrix([[6.29994898, 2.9463052 ],
        [2.9463052 , 5.78507177]])

# Get standardised difference

In [8]:
# Get treatment and control vectors
c = np.matrix(ColPercent['control'])
t = np.matrix(ColPercent['treatment'])
# calculate the mahalanobis distance by defining the vectors for the control and treatment groups
# Calculate the standardised difference 
d = float(np.sqrt((t-c) * S * ((t-c).T)))

d

0.2944457152353158

In [9]:
# Check correct data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   SepalLengthCm  150 non-null    float64
 1   SepalWidthCm   150 non-null    float64
 2   PetalLengthCm  150 non-null    float64
 3   PetalWidthCm   150 non-null    float64
 4   Species        150 non-null    object 
 5   group_var      150 non-null    int32  
dtypes: float64(4), int32(1), object(1)
memory usage: 6.6+ KB


In [10]:
var_names = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm', 'Species']

In [11]:
for col in df[var_names]:
    print(col)
    print(standardised_difference_function(df, treatment_control_flag, col, control_label, treatment_label))
    print("___"*10)

SepalLengthCm
{'standardised_difference': 0.20355772084962398, 'CI': (-0.15626473010179348, 0.5633801718010414), 'p_value': 0.27077564354509354}
______________________________
SepalWidthCm
{'standardised_difference': -0.3124534518084735, 'CI': (-0.6732743852273145, 0.048367481610367524), 'p_value': 1.0187376176225913}
______________________________
PetalLengthCm
{'standardised_difference': 0.22819144144811845, 'CI': (-0.13182020900392002, 0.588203091900157), 'p_value': 0.21592853716551272}
______________________________
PetalWidthCm
{'standardised_difference': 0.1634232057200189, 'CI': (-0.19613706955564003, 0.5229834809956778), 'p_value': 0.37951417124754283}
______________________________
Species
{'standardised_difference': 0.2944457152353158, 'CI': (-0.06618123611150406, 0.6550726665821357), 'p_value': 0.109395282179496}
______________________________
